In [ ]:
!pip install -r requirements.txt
!pip install progress
!pip install pretty_midi
!pip install wandb

In [5]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [8]:
cd /content/drive/'My Drive'/MusicTransformer


/content/drive/My Drive/MusicTransformer


In [9]:
ls

AUTOENCODER/     midi_classical/                 README.md
bin/             midi_prepared_small/            readme_src/
custom/          midi_processed/                 requirements_org.txt
custom_simple/   midi_processed_small/           requirements.txt
data.py          midi_processor/                 simple_test_model.py
dataset/         model_new.py                    test.py
deprecated/      model.py                        train_new.py
dist_train.py    mt-2048-h4-dim256.h5/           train.py
generate.py      music_transformer_train2.ipynb  TRANSFORMER/
hidi_processed/  music-transformer-train.ipynb   utils.py
LICENSE          params.py                       wandb/
logs/            preprocess.py
midi/            __pycache__/


In [10]:

import wandb
wandb.init()

wandb: Currently logged in as: nexon4444 (use `wandb login --relogin` to force relogin)


In [15]:
# from numba import cuda 
# device = cuda.get_current_device()
# device.reset()

In [ ]:

!python preprocess.py midi_classical midi_processed


In [11]:
import os
os.getcwd()

'/content/drive/My Drive/MusicTransformer'

In [12]:
print(os.listdir())

['custom', 'dataset', 'logs', '__pycache__', '.idea', 'midi_processed', 'midi', 'readme_src', 'deprecated', 'midi_classical', 'midi_processor', '.git', 'hidi_processed', '.gitignore', 'LICENSE', 'README.md', 'dist_train.py', 'generate.py', 'preprocess.py', 'utils.py', 'requirements_org.txt', 'requirements.txt', 'midi_prepared_small', '.ipynb_checkpoints', 'mt-2048-h4-dim256.h5', 'params.py', 'midi_processed_small', 'custom_simple', 'bin', 'TRANSFORMER', 'AUTOENCODER', 'model_new.py', 'music_transformer_train2.ipynb', 'simple_test_model.py', 'test.py', 'train_new.py', '.gdriveignore', 'data.py', 'model.py', 'music-transformer-train.ipynb', 'train.py', 'wandb']


In [13]:
from model import MusicTransformer
from custom.layers import *
from custom import callback
from tensorflow.python import keras
# import params as par
import midi_processor.processor as sequence
from tensorflow.python import enable_eager_execution
from tensorflow.python.keras.optimizer_v2.adam import Adam
from data import Data
import utils
# tf.executing_eagerly()

In [14]:
tf.test.is_gpu_available()



Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [25]:
l_r = 0.001 #@param {type:"slider", min:0, max:0.1, step:0.0001}
batch_size = 1 #@param {type:"slider", min:1, max:100, step:1}
pickle_dir = "processed/" #@param {type:"string"}
max_seq = 2048 #@param {type:"slider", min:1, max:3000, step:1}
epochs = 41 #@param {type:"slider", min:1, max:10000, step:1}
model_save_path = "bin/models" #@param {type:"string"}
embedding_dim = 256 #@param {type:"slider", min:2, max:2048, step:1}

event_dim = sequence.RANGE_NOTE_ON + sequence.RANGE_NOTE_OFF + sequence.RANGE_TIME_SHIFT + sequence.RANGE_VEL
vocab_size = event_dim + 3

In [16]:
dataset = Data('dataset/processed/')
opt = Adam(l_r)
mt = MusicTransformer(
    embedding_dim=embedding_dim,
    vocab_size=vocab_size,
    num_layer=3,
    max_seq=max_seq,
    debug=False)
mt.compile(optimizer=opt, loss=callback.TransformerLoss())

ValueError: ignored

# Train Model

In [27]:
!nvidia-smi

Tue Dec 14 14:05:17 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P0    61W / 149W |  10990MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [26]:


import os
from pathlib import Path

from tensorflow.keras.callbacks import ModelCheckpoint

from model_new import MusicTransformer
from custom.layers import *
from custom import callback
import params as par
from tensorflow.keras.optimizers import Adam
from data import DataNew
import utils
import argparse
import datetime

try:
    import google.colab

    IS_ON_GOOGLE_COLAB = True
except:
    IS_ON_GOOGLE_COLAB = False

if IS_ON_GOOGLE_COLAB:
    from google.colab import drive


# tf.executing_eagerly()

num_layer = 6


def get_current_datetime():
    from datetime import datetime
    now = datetime.now()
    dt_name = now.strftime("%m_%d_%Y__%H_%M_%S")
    return dt_name


if IS_ON_GOOGLE_COLAB:
    FOLDER_ROOT = "/content/drive/MyDrive/magisterka/SheetMusicGenerator2"
else:
    FOLDER_ROOT = "."

TEST_RUN = True
NORMALIZE_NOTES = True
USE_COMPUTED_VALUES = True
USE_SAVE_POINT = False

NORMALIZATION_BOUNDARIES = [3, 4]
EPOCHS = 250
LATENT_VECTOR_DIM = 2
BATCH_SIZE = 256
SEQUENCE_LENGTH = 32

FOLDER_ROOT = "."
# COMPUTED_INT_TO_NOTE_PATH = "/content/drive/MyDrive/magisterka/SheetMusicGenerator2/AUTOENCODER/data/dicts/int_to_note_08_19_2021__17_25_44"
# COMPUTED_INT_TO_DURATION_PATH = "/content/drive/MyDrive/magisterka/SheetMusicGenerator2/AUTOENCODER/data/dicts/int_to_duration_08_19_2021__17_25_44"
# COMPUTED_NOTES_PATH = "/content/drive/MyDrive/magisterka/SheetMusicGenerator2/AUTOENCODER/data/notes/notes_08_19_2021__17_25_44"
# COMPUTED_DURATIONS_PATH = "/content/drive/MyDrive/magisterka/SheetMusicGenerator2/AUTOENCODER/data/durations/durations_08_19_2021__17_25_44"
COMPUTED_DATA_PATH = "AUTOENCODER/data/data_file_12_06_2021__19_53_42"

SAVE_POINT = "AUTOENCODER/checkpoints/08_19_2021__18_34_10/epoch=014-loss=383.5284-acc=0.0000.hdf5"
AUTOENCODER = "TRANSFORMER"

MODEL_NAME = AUTOENCODER
MODEL_FOLDER_ROOT = os.path.join(FOLDER_ROOT, MODEL_NAME)
CURR_DT = get_current_datetime()
MODEL_DIR_PATH = os.path.join(MODEL_FOLDER_ROOT, "generated_models")
OCCURENCES = os.path.join(MODEL_FOLDER_ROOT, "data", "occurences")

DATA_DIR = os.path.join(MODEL_FOLDER_ROOT, "data")
DATA_NOTES_DIR = os.path.join(DATA_DIR, "notes")
DATA_DURATIONS_DIR = os.path.join(DATA_DIR, "durations")

DATA_FILE_PATH = os.path.join(DATA_DIR, "data_file_" + str(CURR_DT))

DATA_DICTS_DIR = os.path.join(DATA_DIR, "dicts")
DATA_INT_TO_NOTE_PATH = os.path.join(DATA_DICTS_DIR, "int_to_note_" + str(CURR_DT))
DATA_INT_TO_DURATION_PATH = os.path.join(DATA_DICTS_DIR, "int_to_duration_" + str(CURR_DT))
DATA_NOTES_PATH = os.path.join(DATA_NOTES_DIR, "notes_" + str(CURR_DT))

DATA_DURATIONS_PATH = os.path.join(DATA_DURATIONS_DIR, "durations_" + str(CURR_DT))
# MIDI_SONGS_DIR = os.path.join(FOLDER_ROOT, "midi_songs")
MIDI_SONGS_DIR = os.path.join(FOLDER_ROOT, "midi_songs_smaller")
# MIDI_SONGS_DIR = os.path.join(FOLDER_ROOT, "midi_songs_medium")
MIDI_GENERATED_DIR = os.path.join(MODEL_FOLDER_ROOT, "midi_generated")
MIDI_SONGS_REGEX = os.path.join(MIDI_SONGS_DIR, "*.mid")
CHECKPOINTS_DIR = os.path.join(MODEL_FOLDER_ROOT, "checkpoints")
CHECKPOINT = os.path.join(CHECKPOINTS_DIR, str(CURR_DT))
LOGS_DIR = os.path.join(MODEL_FOLDER_ROOT, "logs")

LOG = os.path.join(LOGS_DIR, str(CURR_DT))
all_paths = [MODEL_DIR_PATH, OCCURENCES, DATA_NOTES_DIR, DATA_DURATIONS_DIR, DATA_DICTS_DIR,
             MIDI_GENERATED_DIR, CHECKPOINTS_DIR, CHECKPOINT, LOGS_DIR, LOG]

for path in all_paths:
    Path(path).mkdir(parents=True, exist_ok=True)
# load data
dataset = DataNew('midi_processed', max_seq, batch_size)
# print(dataset)


# load model
curr_dt = get_current_datetime()
learning_rate = callback.CustomSchedule(par.embedding_dim) if l_r is None else l_r
opt = Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)
filepath = CHECKPOINT + str(curr_dt) + "/" + "epoch:{epoch:02d}-loss:{loss:.4f}" #-acc:{binary_accuracy:.4f}.hdf5"
checkpoint = ModelCheckpoint(
    filepath,
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='max'
)
#
# checkpoint = ModelCheckpoint(
#     filepath,
#     monitor='binary_accuracy',
#     verbose=0,
#     save_best_only=True,
#     mode='max'
# )
log = tf.keras.callbacks.TensorBoard(log_dir=LOG + curr_dt),

# callbacks_list = [checkpoint, log]
callbacks_list = [log]
# define model
mt = MusicTransformer(
            embedding_dim=256,
            vocab_size=par.vocab_size,
            num_layer=num_layer,
            max_seq=max_seq,
            dropout=0.2,
            debug=False, loader_path=None)
mt.compile(optimizer=opt, loss=callback.transformer_dist_train_loss)
mt.run_eagerly = True
# batch = (dataset.generators_dict["train"][0])
# print(type(dataset.generators_dict["train"]))
# mt.train_on_batch(batch[0], batch[1])
print(len(dataset.generators_dict["train"][0]))
print(len(dataset.generators_dict["train"][1]))
print(len(dataset.generators_dict["train"][2]))
# ds = tf.data.Dataset.from_generator(dataset.generators_dict["train"].__iter__(), output_types=tf.float32)
mt.fit(x=dataset.generators_dict["train"][0], y=dataset.generators_dict["train"][1], epochs=EPOCHS, callbacks=callbacks_list)
# mt.fit(dataset.generators_dict["train"], epochs=EPOCHS, callbacks=callbacks_list)
# mt.fit(dataset.generators_dict["train"], epochs=EPOCHS, callbacks=callbacks_list)

# define tensorboard writer
current_time = datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
train_log_dir = 'logs/mt_decoder/'+current_time+'/train'
eval_log_dir = 'logs/mt_decoder/'+current_time+'/eval'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
eval_summary_writer = tf.summary.create_file_writer(eval_log_dir)
EPOCHS=2

# Train Start
# history = mt.fit(dataset.slide_seq2seq_batch(batch_size, max_seq), epochs=EPOCHS)

# idx = 0
# for e in range(epochs):
#     mt.reset_metrics()
#     for b in range(len(dataset.files) // batch_size):
#         try:
#             batch_x, batch_y = dataset.slide_seq2seq_batch(batch_size, max_seq)
#         except:
#             continue
#         result_metrics = mt.train_on_batch(batch_x, batch_y)
#         if b % 100 == 0:
#             eval_x, eval_y = dataset.slide_seq2seq_batch(batch_size, max_seq, 'eval')
#             eval_result_metrics, weights = mt.evaluate(eval_x, eval_y)
#             mt.save(save_path)
#             with train_summary_writer.as_default():
#                 if b == 0:
#                     tf.summary.histogram("target_analysis", batch_y, step=e)
#                     tf.summary.histogram("source_analysis", batch_x, step=e)
#
#                 tf.summary.scalar('loss', result_metrics[0], step=idx)
#                 tf.summary.scalar('accuracy', result_metrics[1], step=idx)
#
#             with eval_summary_writer.as_default():
#                 if b == 0:
#                     mt.sanity_check(eval_x, eval_y, step=e)
#
#                 tf.summary.scalar('loss', eval_result_metrics[0], step=idx)
#                 tf.summary.scalar('accuracy', eval_result_metrics[1], step=idx)
#                 for i, weight in enumerate(weights):
#                     with tf.name_scope("layer_%d" % i):
#                         with tf.name_scope("w"):
#                             utils.attention_image_summary(weight, step=idx)
#                 # for i, weight in enumerate(weights):
#                 #     with tf.name_scope("layer_%d" % i):
#                 #         with tf.name_scope("_w0"):
#                 #             utils.attention_image_summary(weight[0])
#                 #         with tf.name_scope("_w1"):
#                 #             utils.attention_image_summary(weight[1])
#             idx += 1
#             print('\n====================================================')
#             print('Epoch/Batch: {}/{}'.format(e, b))
#             print('Train >>>> Loss: {:6.6}, Accuracy: {}'.format(result_metrics[0], result_metrics[1]))
#             print('Eval >>>> Loss: {:6.6}, Accuracy: {}'.format(eval_result_metrics[0], eval_result_metrics[1]))


x: $[[366  65 366 ...  77 367  65]]
2
x: $[[370  60 266 ... 371  72 371]]
2
x: $[[370  62 374 ... 186 210 367]]
2
x: $[[230 218 225 ... 369  70 369]]
x: $[[366  60 283 ... 373  77 264]]
Epoch 1/250
targets: [[218 225 295 ...  70 369  66]]
inputs: [[230 218 225 ... 369  70 369]]
type(x): <class 'tensorflow.python.framework.ops.EagerTensor'>
x: [[230 218 225 ... 369  70 369]]
<class 'tensorflow.python.framework.ops.EagerTensor'>
out2.shape: (1, 2048, 256)
w.shape: (1, 4, 2048, 2048)
out2.shape: (1, 2048, 256)
w.shape: (1, 4, 2048, 2048)
out2.shape: (1, 2048, 256)
w.shape: (1, 4, 2048, 2048)
out2.shape: (1, 2048, 256)
w.shape: (1, 4, 2048, 2048)
out2.shape: (1, 2048, 256)
w.shape: (1, 4, 2048, 2048)
out2.shape: (1, 2048, 256)
w.shape: (1, 4, 2048, 2048)
1/1 [==============================] - 3s 3s/step - loss: 6.5308 - sparse_categorical_accuracy: 0.0000e+00
Epoch 2/250
targets: [[218 225 295 ...  70 369  66]]
inputs: [[230 218 225 ... 369  70 369]]
type(x): <class 'tensorflow.python.fram

ResourceExhaustedError: ignored

In [ ]:
# for e in range(epochs):
#     for b in range(len(dataset.files) // batch_size):
#         batch_x, batch_y = dataset.seq2seq_batch(batch_size, par.max_seq)
#         result_metrics = mt.train_on_batch(batch_x, batch_y)
#         if b % 1 == 0:
#           print('===========================================\n')
#           print('Loss: {:6.6}, Accuracy: {:3.2}'.format(result_metrics[0], result_metrics[1]))
#           mt.save('mt-2048-h4-dim256.h5', )



In [ ]:
MusicTransformer

In [ ]:
!python train.py --epochs=1 --pickle_dir="dataset/processed" --save_path="bin/models" --max_seq=2048  --batch_size=1 --l_r=0.001

# Generate Model

In [ ]:
def fill_with_placeholder(prev_data: list, max_len: int, max_val: int=239):
    placeholder = [max_val for _ in range(max_len - len(prev_data))]
    return placeholder+prev_data

In [ ]:
for i in range(max_seq):


In [ ]:
from keras import models
from keras import layers
from keras.datasets import mnist
from keras.utils import np_utils
import matplotlib.pyplot as plt
import numpy as np

# suppress warning/error messages in terminal
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# load in data from mnist dataset (60k training, 10k test)
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# reshape and convert to one-hot
x_train = x_train.reshape(x_train.shape[0], 784)
x_test = x_test.reshape(x_test.shape[0], 784)
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

# split main training set into train/validation sets (6k out of 60k data points reserved for validation)
x_validate = x_train[:6000, :]
y_validate = y_train[:6000, :]
x_train = x_train[6000:, :]
y_train = y_train[6000:, :]

'''
Building layers for the feedforward NN:
Hidden layers have 56, 96, 96, and 56 nodes, in that order.
Takes in 784 values (pixel input) and outputs 10 values (predicted probability for each number, 0-9).
'''
model = models.Sequential()
model.add(layers.Dense(56,activation='relu',input_shape=(784,)))
model.add(layers.Dense(96,activation='relu'))
model.add(layers.Dense(96,activation='relu'))
model.add(layers.Dense(56,activation='relu'))
model.add(layers.Dense(10,activation='softmax'))

model.compile(loss='categorical_crossentropy',
			  optimizer='adam',
			  metrics=['accuracy'])

# training the NN
epochs = 20
history = model.fit(x_train,y_train,epochs=epochs,batch_size=512,verbose=2,validation_data=(x_validate,y_validate))
print("Finished fitting.")

# plotting learning curves during training (on both training and validation data)
epoch_labels = range(1, epochs+1)
hist_dict = history.history
plt.title("Accuracy vs Epochs")
plt.plot(epoch_labels, hist_dict["acc"],'bo', label="Training")
plt.plot(epoch_labels, hist_dict["val_acc"],'go', label="Validation")
plt.legend(loc="best")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")

# evaluating final performance using test data
print("Checking accuracy on test set...")
acc = model.evaluate(x_test,y_test,batch_size=512)
print("\nAccuracy on test set: " + str(acc[1]))
plt.show()

In [23]:
import gc
gc.collect()
tf.keras.backend.clear_session()